# Load Packages

In [1]:
import io
import json
import numpy as np
import pickle
import unidecode
import re
import sys

# Fix seed
np.random.seed(100)

# Preprocessing

In [2]:
# Regex for removing all non-alphabet letters or spaces
regex = re.compile('[^a-zA-Z ]')

def preprocessing(documents):
    for i in range(len(documents)):
        # Remove all accents
        documents[i] = unidecode.unidecode(documents[i])
        
        # Remove all non-alphabet letters or spaces
        documents[i] = regex.sub(' ', documents[i])
        
        # Remove extra spaces
        documents[i] = ' '.join([token for token in documents[i].split(' ') if token])
        
        # To lower
        documents[i] = documents[i].lower()
    return documents

# Finding Similar Items

## Utils

### Universal Hashing

In [3]:
def universal_hash(prime, size):
    # Check values
    if prime < size:
        raise ValueError("Prime number should be greater than size")
    
    # Generate random values
    a = np.random.randint(1, prime)
    b = np.random.randint(1, prime)
    
    # Return hash function
    return lambda x: ((a * x + b) % prime) % size

## Shingling

In [4]:
def to_shingles(documents, k):
    # Map keeping the "Hash" of the shingles
    shingle_map = {}
    idx = 0
    
    # New document structure
    doc_shingles = []
    for i, document in enumerate(documents):
        shingles = set()
        # Split each document in k-shingles
        for j in range(0, len(document) - k + 1):
            # Get shingle
            shingle = document[j:j+k]
            
            # For efficience purposes, apply hash
            if shingle in shingle_map:
                hashed_shingle = shingle_map[shingle]
            else:
                shingle_map[shingle] = idx
                hashed_shingle = idx
                idx += 1
            
            # Append to set of shingles
            shingles.add(hashed_shingle)
            
        # Attribute to document
        doc_shingles.append(shingles)
        
    return doc_shingles, shingle_map

## Min-Hashing

In [5]:
def compute_min_hashing(documents, shingles_size, k, prime=2**61-1):
    # Instantiate hash methods to be used as permutations
    hash_methods = [universal_hash(prime, shingles_size)
                    for i in range(k)]
    
    # Signature of each document
    signatures = [[sys.maxsize
                   for j in range(k)]
                  for i in range(len(documents))]
    # signatures = np.full((len(documents), k), np.Inf, dtype=np.int32)
    
    # Each shingle for each document just need to be computed once
    computed = [set() for i in range(len(documents))]
    
    for i, document in enumerate(documents):
        for shingle in document:
            # Shingle already computed
            if shingle in computed[i]:
                continue
            
            # Compute hash for shingle
            computed[i].add(shingle)
            for j, hash_method in enumerate(hash_methods):
                hash_value = hash_method(shingle)
                
                # Check if "permutation position" is lower
                if hash_value < signatures[i][j]:
                    signatures[i][j] = hash_value
    
    # Return signature of all documents
    return np.array(signatures, dtype=np.uint64)

## Locality-Sensitive Hashing

In [6]:
def compute_lsh(signatures, rows, bands, prime=2**61-1):
    # Make n_buckets as large as possible
    # For now, we will use "1GB"
    n_buckets = int(10**9)
    
    # Instantiate hash methods
    hash_methods = [universal_hash(prime, n_buckets)
                    for i in range(bands)]
    
    # Buckets for all hashes
    hash_buckets = [{} for i in range(bands)]
    
    for i, signature in enumerate(signatures):
        for j in range(bands):
            # Get mini signature
            mini_signature = signature[j*rows:j*rows+rows]
            
            # "Merge" entries of vector
            value = np.sum(np.power(mini_signature, 2))
                
            # Compute hash/bucket for the band
            hash_value = hash_methods[j](value)

            if hash_value in hash_buckets[j]:
                hash_buckets[j][hash_value].append(i)
            else:
                hash_buckets[j][hash_value] = [i]
    
    return hash_buckets

## Find Candidates

In [7]:
def find_candidates(hash_buckets, signatures, threshold):
    # Get signature size - signatures y-axis
    signature_size = signatures.shape[1]
    
    # Get all pairs
    pairs = set()
    for hash_bucket in hash_buckets:
        for bucket, values in hash_bucket.items():
            # Only interested in pairs
            if len(values) < 2:
                continue
            
            # Check if items are candidates (> threshold)
            for i in range(0, len(values)):
                for j in range(i + 1, len(values)):
                    # Keep order, so we can eliminate duplicates
                    if values[i] > values[j]:
                        pairs.add((values[j], values[i]))
                    else:
                        pairs.add((values[i], values[j]))
    
    # Find all candidates                        
    candidates = []
    for pair in pairs:
        idx, idy = pair
        equal_values = np.sum(signatures[idx] == signatures[idy])
        if equal_values >= threshold * signature_size:
            candidates.append(pair)
    
    return candidates

In [54]:
def set_clusters(candidates, n_items):
    # Initialize clusters
    cids = {i:i for i in range(n_items)}
    clusters = {i:set([i]) for i in range(n_items)}
    
    # Fill clusters
    for item_a, item_b in candidates:
        # Already in the same cluster due to composition
        if cids[item_a] == cids[item_b]:
            continue
            
        # Get current clusters
        cluster_a = clusters[cids[item_a]]
        cluster_b = clusters[cids[item_b]]
        
        # Merge clusters
        if len(cluster_a) >= len(cluster_b):
            new_cid = cids[item_a]
            old_cid = cids[item_b]
        else:
            new_cid = cids[item_b]
            old_cid = cids[item_a]

        # Update
        for item in clusters[old_cid]:
            cids[item] = new_cid
        clusters[new_cid].update(clusters[old_cid])
        del clusters[old_cid]
        
    return clusters, cids

# Experiments

## Parameters

In [8]:
data_path = "../datasets/development.json"
dump_path = "../datasets/cids.tsv"

## Hyperparameters

In [9]:
k = 5
n_rows = 20
n_bands = 5
n_hashes = n_rows * n_bands
threshold = 0.999

## Load dataset

In [10]:
data_reader = io.open(data_path, mode="r", encoding="utf-8")

# Go to beginning
data_reader.seek(0)

# Parse all text from json
documents = [document['description']
             for document in json.loads(data_reader.readline())
             if document['description']]

data_reader.close()

## Run

In [11]:
%time parsed_documents = preprocessing(documents)

CPU times: user 2.22 s, sys: 4 ms, total: 2.23 s
Wall time: 2.23 s


In [12]:
%time documents_shingles, map_shingles = to_shingles(parsed_documents, k)

CPU times: user 3.34 s, sys: 68 ms, total: 3.4 s
Wall time: 3.4 s


In [13]:
%time signatures = compute_min_hashing(documents_shingles, len(map_shingles), n_hashes)

CPU times: user 5min 48s, sys: 136 ms, total: 5min 48s
Wall time: 5min 48s


In [14]:
%time hash_buckets = compute_lsh(signatures, n_rows, n_bands)

CPU times: user 1.12 s, sys: 4 ms, total: 1.12 s
Wall time: 1.12 s


In [15]:
%time candidates = find_candidates(hash_buckets, signatures, threshold)

CPU times: user 120 ms, sys: 4 ms, total: 124 ms
Wall time: 126 ms


In [55]:
%time clusters, cids = set_clusters(candidates, len(documents))

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 20.6 ms


## Dump cluster ids

In [88]:
with open(dump_path, 'w') as fp:
    for cid in cids.items():
        fp.write('%s\t%s\n' % cid)

## Analysis

In [16]:
len(map_shingles)

77050

In [17]:
len(candidates)

3305

In [18]:
for i in range(2):
    idx = np.random.randint(0, len(candidates))
    item_a = candidates[idx][0]
    item_b = candidates[idx][1]
    print('%s:\t%s' % (item_a, documents[item_a]))
    print('%s:\t%s' % (item_b, documents[item_b]))
    print()

1223:	vaga encarregado pavimentacao paulinia spliderar equipe acompanhar e checar as escavacoes em execucao analisar a umidade e qualidade de materiais para compactacao do solo coordenar a compactacao requisitos residir em campinas hortolndia sumare paulinia experiencia comprovada na funcao beneficios vale transporte ou auxilio combustivel refeicao no local cesta basica convenio medico e odontologico seguro de vida horario de trabalho de a a a das as horas
4679:	vaga encarregado pavimentacao paulinia spliderar equipe acompanhar e checar as escavacoes em execucao analisar a umidade e qualidade de materiais para compactacao do solo coordenar a compactacao requisitos residir em campinas hortolndia sumare paulinia experiencia comprovada na funcao beneficios vale transporte ou auxilio combustivel refeicao no local cesta basica convenio medico e odontologico seguro de vida horario de trabalho de a a a das as horas

1223:	vaga encarregado pavimentacao paulinia spliderar equipe acompanhar e ch